In [67]:
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


READ AND PREPROCESS DATA

In [68]:
df =pd.read_csv("output.csv")

In [69]:
df.shape

(2832, 2)

In [70]:
df.head()

,Bot_Cell,Crew_Cell
0,"(2, 8)","(3, 9)"
1,"(2, 9)","(3, 8)"
2,"(3, 9)","(4, 8)"
3,"(4, 9)","(5, 8)"
4,"(5, 9)","(5, 7)"


In [71]:
# Extract x, y, p, and q values from the existing columns
df['x'] = df['Bot_Cell'].apply(lambda x: int(x.split(',')[0].strip('()')))
df['y'] = df['Bot_Cell'].apply(lambda x: int(x.split(',')[1].strip('()')))
df['p'] = df['Crew_Cell'].apply(lambda x: int(x.split(',')[0].strip('()')))
df['q'] = df['Crew_Cell'].apply(lambda x: int(x.split(',')[1].strip('()')))

# Calculate the distance
df['Distance_from_bot_to_crew'] = abs(df['x'] - df['p']) + abs(df['y'] - df['q'])
df['Distance_from_bot_to_teleport'] = abs(df['x'] - 5) + abs(df['y'] - 5)
df['Distance_from_crew_to_teleport'] = abs(5 - df['p']) + abs(5 - df['q'])
#Drop the intermediate columns x, y, p, and q if needed
df.drop(['x', 'y', 'p', 'q'], axis=1, inplace=True)



In [72]:
df['Bot_Move'] = df['Bot_Cell'].shift(-1)
df.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move
0,"(2, 8)","(3, 9)",2,6,6,"(2, 9)"
1,"(2, 9)","(3, 8)",2,7,5,"(3, 9)"
2,"(3, 9)","(4, 8)",2,6,4,"(4, 9)"
3,"(4, 9)","(5, 8)",2,5,3,"(5, 9)"
4,"(5, 9)","(5, 7)",2,4,2,"(5, 8)"


In [74]:
def calculate_angle(point1, point2):
    origin = (5, 5)  # Assuming (5, 5) as the point of origin
    vector1 = (point1[0] - origin[0], point1[1] - origin[1])
    vector2 = (point2[0] - origin[0], point2[1] - origin[1])
    
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude1 = math.sqrt(vector1[0]**2 + vector1[1]**2)
    magnitude2 = math.sqrt(vector2[0]**2 + vector2[1]**2)
    
    if magnitude1 == 0 or magnitude2 == 0:
        return 0  # Return 0 angle if magnitude is zero
    
    angle_radians = math.acos(dot_product / (magnitude1 * magnitude2))
    angle_degrees = math.degrees(angle_radians)
    
    return angle_degrees

def parse_tuple(string_tuple):
    return tuple(map(int, string_tuple.strip('()').split(',')))

def calculate_angle_for_row(row):
    bot_cell = parse_tuple(row['Bot_Cell'])
    crew_cell = parse_tuple(row['Crew_Cell'])
    return calculate_angle(bot_cell, crew_cell)
    


In [75]:
df['Angle'] = df.apply(calculate_angle_for_row, axis=1)

In [76]:
def parse_coordinates(coord_str):
    if coord_str:
        x, y = map(int, coord_str.strip("()").split(","))
        return x, y
    else:
        return None

def calculate_direction(row):
    bot_cell = parse_coordinates(row['Bot_Cell'])
    bot_move = parse_coordinates(row['Bot_Move'])

    if bot_cell and bot_move:
        delta_x = bot_move[0] - bot_cell[0]
        delta_y = bot_move[1] - bot_cell[1]

        if delta_x == 0 and delta_y == 0:
            return "No movement"
        elif delta_x == 0:
            return "North" if delta_y > 0 else "South"
        elif delta_y == 0:
            return "East" if delta_x > 0 else "West"
        elif delta_x > 0:
            return "Northeast" if delta_y > 0 else "Southeast"
        else:
            return "Northwest" if delta_y > 0 else "Southwest"
    else:
        return "Invalid coordinates"

# Apply the calculate_direction function to each row and store the result in a new column
df['Direction_of_Bot'] = df.apply(lambda row: calculate_direction(row), axis=1)
    

In [77]:
df.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move,Angle,Direction_of_Bot
0,"(2, 8)","(3, 9)",2,6,6,"(2, 9)",18.434949,North
1,"(2, 9)","(3, 8)",2,7,5,"(3, 9)",3.179830,East
2,"(3, 9)","(4, 8)",2,6,4,"(4, 9)",8.130102,East
3,"(4, 9)","(5, 8)",2,5,3,"(5, 9)",14.036243,East
4,"(5, 9)","(5, 7)",2,4,2,"(5, 8)",0.000000,South


Encodding the data

In [78]:
one_hot_encoded_df = df.copy()
one_hot_encoded_df = pd.get_dummies(df)
one_hot_encoded_df.head()

,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Angle,"Bot_Cell_(0, 0)","Bot_Cell_(0, 1)","Bot_Cell_(0, 10)","Bot_Cell_(0, 2)","Bot_Cell_(0, 3)","Bot_Cell_(0, 4)",...,Direction_of_Bot_East,Direction_of_Bot_Invalid coordinates,Direction_of_Bot_No movement,Direction_of_Bot_North,Direction_of_Bot_Northeast,Direction_of_Bot_Northwest,Direction_of_Bot_South,Direction_of_Bot_Southeast,Direction_of_Bot_Southwest,Direction_of_Bot_West
0,2,6,6,18.434949,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,2,7,5,3.179830,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,2,6,4,8.130102,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,2,5,3,14.036243,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,2,4,2,0.000000,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [79]:
label_encoder = LabelEncoder()
label_encoded_df = df.copy()
# Iterate over each column in the DataFrame
for column in label_encoded_df.columns:
    # Check if the column is categorical (i.e., dtype is object)
    if label_encoded_df[column].dtype == 'object':
        # Fit label encoder and transform values for the current column
        label_encoded_df[column] = label_encoder.fit_transform(label_encoded_df[column])

In [80]:
label_encoded_df.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move,Angle,Direction_of_Bot
0,42,54,2,6,6,43,18.434949,3
1,43,53,2,7,5,54,3.179830,0
2,54,62,2,6,4,63,8.130102,0
3,63,73,2,5,3,73,14.036243,0
4,73,72,2,4,2,72,0.000000,6


Scalling the data

In [81]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Assuming your data is stored in a DataFrame called df

# Select the columns you want to scale
columns_to_scale = ['Bot_Cell', 'Crew_Cell', 'Distance_from_bot_to_crew', 'Distance_from_crew_to_teleport', 'Bot_Move', 'Angle',"Direction_of_Bot","Distance_from_bot_to_teleport"]

# Initialize the scaler
scaler = MinMaxScaler()  # You can also use StandardScaler if you prefer standardization

# Fit and transform the selected columns
scaled_data = scaler.fit_transform(label_encoded_df[columns_to_scale])

# Convert the scaled data back to a DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)

# Now you can use the scaled_df DataFrame for further analysis or modeling


In [87]:
scaled_df.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_crew_to_teleport,Bot_Move,Angle,Direction_of_Bot,Distance_from_bot_to_teleport
0,0.365217,0.465517,0.0625,0.6,0.370690,0.102416,0.333333,0.555556
1,0.373913,0.456897,0.0625,0.5,0.465517,0.017666,0.000000,0.666667
2,0.469565,0.534483,0.0625,0.4,0.543103,0.045167,0.000000,0.555556
3,0.547826,0.629310,0.0625,0.3,0.629310,0.077979,0.000000,0.444444
4,0.634783,0.620690,0.0625,0.2,0.620690,0.000000,0.666667,0.333333


In [82]:

correlation_matrix2 = scaled_df.corr()
correlation_matrix1 = one_hot_encoded_df.corr()

In [56]:
correlation_matrix1

,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Angle,"Bot_Cell_(0, 0)","Bot_Cell_(0, 1)","Bot_Cell_(0, 10)","Bot_Cell_(0, 2)","Bot_Cell_(0, 3)","Bot_Cell_(0, 4)",...,Direction_of_Bot_East,Direction_of_Bot_Invalid coordinates,Direction_of_Bot_No movement,Direction_of_Bot_North,Direction_of_Bot_Northeast,Direction_of_Bot_Northwest,Direction_of_Bot_South,Direction_of_Bot_Southeast,Direction_of_Bot_Southwest,Direction_of_Bot_West
Distance_from_bot_to_crew,1.000000,-0.053639,0.288303,0.858104,0.000254,-0.022082,0.042786,-0.015093,0.011732,0.003060,...,-0.097745,-0.005888,-0.079497,-0.119481,0.187132,0.117100,-0.089368,0.163006,0.218124,-0.083493
Distance_from_bot_to_teleport,-0.053639,1.000000,0.562539,-0.165952,0.122377,0.164375,0.134081,0.128010,0.099215,0.054168,...,0.079483,-0.032188,-0.011856,0.066517,-0.097820,-0.113543,0.071709,-0.085071,-0.099085,0.028310
Distance_from_crew_to_teleport,0.288303,0.562539,1.000000,0.184983,0.065686,0.106075,0.078094,0.070325,0.049748,0.025236,...,-0.055350,-0.039971,0.020838,-0.049414,0.052779,0.011649,-0.006600,0.050363,0.068505,-0.005975
Angle,0.858104,-0.165952,0.184983,1.000000,-0.011016,-0.036862,0.014807,-0.022828,-0.003063,-0.018181,...,-0.097428,-0.015990,-0.060707,-0.128381,0.154351,0.148756,-0.086437,0.146577,0.203360,-0.063564
"Bot_Cell_(0, 0)",0.000254,0.122377,0.065686,-0.011016,1.000000,-0.002964,-0.001938,-0.003069,-0.003547,-0.003805,...,0.044376,-0.000790,-0.005919,0.022036,-0.010154,-0.010188,-0.019473,-0.010826,-0.011150,-0.019992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Direction_of_Bot_Northwest,0.117100,-0.113543,0.011649,0.148756,-0.010188,-0.017076,-0.011163,-0.017678,-0.020431,-0.021922,...,-0.116514,-0.004553,-0.034094,-0.118533,-0.058493,1.000000,-0.112178,-0.062363,-0.064232,-0.115164
Direction_of_Bot_South,-0.089368,0.071709,-0.006600,-0.086437,-0.019473,-0.019433,0.059228,-0.021030,-0.027991,-0.021263,...,-0.222696,-0.008703,-0.065165,-0.226554,-0.111800,-0.112178,1.000000,-0.119196,-0.122767,-0.220115
Direction_of_Bot_Southeast,0.163006,-0.085071,0.050363,0.146577,-0.010826,0.002710,0.019949,0.021517,0.013223,-0.023293,...,-0.123804,-0.004838,-0.036227,-0.125948,-0.062153,-0.062363,-0.119196,1.000000,-0.068250,-0.122369
Direction_of_Bot_Southwest,0.218124,-0.099085,0.068505,0.203360,-0.011150,-0.018688,-0.012217,-0.019347,-0.022360,-0.023991,...,-0.127513,-0.004983,-0.037313,-0.129722,-0.064015,-0.064232,-0.122767,-0.068250,1.000000,-0.126035


In [83]:
correlation_matrix2

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_crew_to_teleport,Bot_Move,Angle,Direction_of_Bot,Distance_from_bot_to_teleport
Bot_Cell,1.000000,0.613524,0.038463,-0.112924,0.815584,0.066574,0.181750,-0.248149
Crew_Cell,0.613524,1.000000,-0.052861,-0.230329,0.723479,-0.040194,-0.063453,-0.130161
Distance_from_bot_to_crew,0.038463,-0.052861,1.000000,0.288303,0.110175,0.858104,0.113806,-0.053639
Distance_from_crew_to_teleport,-0.112924,-0.230329,0.288303,1.000000,-0.095021,0.184983,0.062166,0.562539
Bot_Move,0.815584,0.723479,0.110175,-0.095021,1.000000,0.107045,0.028547,-0.162879
Angle,0.066574,-0.040194,0.858104,0.184983,0.107045,1.000000,0.120248,-0.165952
Direction_of_Bot,0.181750,-0.063453,0.113806,0.062166,0.028547,0.120248,1.000000,-0.071406
Distance_from_bot_to_teleport,-0.248149,-0.130161,-0.053639,0.562539,-0.162879,-0.165952,-0.071406,1.000000


TRAIN TEST SPLIT

In [84]:
encoded_data1 = one_hot_encoded_df.copy()
encoded_data2 = label_encoded_df.copy()
final_data = scaled_df.copy()
raw_data = df.copy()

In [60]:
X = final_data.drop('Bot_Move', axis=1)
y = final_data['Bot_Move']

In [61]:
X.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_crew_to_teleport,Angle,Direction_of_Bot
0,0.365217,0.465517,0.0625,0.6,0.102416,0.333333
1,0.373913,0.456897,0.0625,0.5,0.017666,0.000000
2,0.469565,0.534483,0.0625,0.4,0.045167,0.000000
3,0.547826,0.629310,0.0625,0.3,0.077979,0.000000
4,0.634783,0.620690,0.0625,0.2,0.000000,0.666667


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
X_train

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_crew_to_teleport,Angle,Direction_of_Bot
2741,0.008696,0.120690,0.0625,0.7,0.009944,0.333333
2346,0.878261,0.525862,0.2500,0.3,0.545167,0.555556
2386,0.652174,0.551724,0.0625,0.5,0.077979,0.666667
2201,0.513043,0.577586,0.0625,0.3,0.147584,0.888889
786,0.765217,0.784483,0.0625,0.3,0.295167,0.444444
...,...,...,...,...,...,...
1638,0.982609,0.577586,0.5000,0.3,0.647584,0.888889
1095,0.921739,0.732759,0.0625,0.6,0.102416,1.000000
1130,0.417391,0.431034,0.0625,0.2,0.250000,0.333333
1294,0.086957,0.068966,0.0625,0.7,0.093658,0.666667


MODELS

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming 'df' is your DataFrame containing the data
X = final_data[['Bot_Cell', 'Crew_Cell', 'Distance_from_bot_to_crew', 'Distance_from_bot_to_teleport', 'Distance_from_crew_to_teleport', 'Angle', 'Direction_of_Bot']]
y = final_data['Bot_Move']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing data
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.017540560045188235


ValueError: continuous is not supported